<a href="https://colab.research.google.com/github/Musa-Ali-Kazmi/Hand-drawn-Sketch-recognition-using-CNN/blob/main/SketchDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!unrar x /content/drive/MyDrive/DL-Assignment2/Train.rar /content/drive/MyDrive/DL-Assignment2


In [ ]:
!unrar x /content/drive/MyDrive/DL-Assignment2/Validation.rar /content/drive/MyDrive/DL-Assignment2

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [9]:
# Load the CSV file containing image filenames and labels
train_df = pd.read_csv('/content/drive/MyDrive/DL-Assignment2/Train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/DL-Assignment2/Validation.csv')

# Define ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=20,  # Random rotation up to 20 degrees
    width_shift_range=0.2,  # Random shift width up to 20% of the image width
    height_shift_range=0.2,  # Random shift height up to 20% of the image height
    shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction in degrees)
    zoom_range=0.2,  # Random zoom up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

train_df['Image'] = train_df['Image'].astype(str)  # Convert values in 'Image' column to strings
val_df['Image'] = val_df['Image'].astype(str)  # Convert values in 'Image' column to strings

train_df['Label'] = train_df['Label'].astype(str)  # Convert values in 'Label' column to strings
val_df['Label'] = val_df['Label'].astype(str)  # Convert values in 'Label' column to strings

# Assuming 'x_column_name' is the name of the column containing the filenames
train_df['Image'] = train_df['Image'].apply(lambda x: str(x) + '.png')
val_df['Image'] = val_df['Image'].apply(lambda x: str(x) + '.png')


# Flow images from the "Train" folder using the ImageDataGenerator and dataframe
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/drive/MyDrive/DL-Assignment2/Train',  # Path to the directory containing the images
    x_col="Image",  # Column containing the image filenames
    y_col="Label",  # Column containing the image labels
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=64,  # Batch size
    class_mode='categorical'  # Type of label encoding
  )

# Flow images from the "Train" folder using the ImageDataGenerator and dataframe
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='/content/drive/MyDrive/DL-Assignment2/Validation',  # Path to the directory containing the images
    x_col="Image",  # Column containing the image filenames
    y_col="Label",  # Column containing the image labels
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=64,  # Batch size
    class_mode='categorical'  # Type of label encoding
  )

Found 10000 validated image filenames belonging to 250 classes.
Found 5000 validated image filenames belonging to 250 classes.


In [4]:
# Load ResNet-50 pre-trained on ImageNet without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


In [5]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create your model on top of ResNet50
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(250, activation='softmax')  # Example output classes, adjust as needed
])

In [6]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048   23587712  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 250)               64250     
                                                                 
Total params: 24176506 (92.23 MB)
Trainable params: 588794 (2.25 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
# Train the model using the train_generator
history = model.fit(
                              train_generator,
                              steps_per_epoch=len(train_generator),
                              epochs=10,
                              validation_data=val_generator,
                              validation_steps=len(val_generator))

Epoch 1/10
157/157 [==============================] - 5701s 36s/step - loss: 5.5017 - accuracy: 0.0058 - val_loss: 5.4606 - val_accuracy: 0.0096
Epoch 2/10
 38/157 [======>.......................] - ETA: 2:39 - loss: 5.4482 - accuracy: 0.0049